In [1]:
from datetime import datetime, timedelta
from pathlib import Path

import itertools
import matplotlib
from vivarium import Artifact, InteractiveContext
import ipywidgets
import pandas as pd, numpy as np
import seaborn as sns
pd.set_option('display.max_rows', 8)

In [2]:
import pickle
from vivarium_ciff_sam.constants import data_keys, metadata

The purpose of this notebook is to verify:

1. if the age-specific state table relative risks are equivalent to the pipeline relative risks
2. if the pipeline relative risk updates appropriately upon transition from the early to late neonatal age group
3. if the pipeline relative risk value does not change across timesteps other than for the age group transition

In [3]:
path = Path('/ihme/homes/alibow/vivarium_ciff_sam/src/vivarium_ciff_sam/model_specifications/ciff_sam.yaml')
path

PosixPath('/ihme/homes/alibow/vivarium_ciff_sam/src/vivarium_ciff_sam/model_specifications/ciff_sam.yaml')

In [4]:
# manually updated the following parameters in the model specification file because it threw an error otherwise
    # population_size=100000
    # age_end=0.0767123287671233
    # exit_age=0.0767123287671233

sim = InteractiveContext(Path(path))

2022-03-04 15:33:13.930 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering metrics.1.population_manager.metrics as modifier to metrics
2022-03-04 15:33:14.041 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:65 - Running simulation from artifact located at /ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf.
2022-03-04 15:33:14.043 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:66 - Artifact base filter terms are ['draw == 0'].
2022-03-04 15:33:14.045 | DEBUG    | vivarium.framework.artifact.manager:_load_artifact:67 - Artifact additional filter terms are None.
/ihme/homes/alibow/vivarium_public_health/src/vivarium_public_health/population/data_transformations.py:441: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  birth_data[birth_data.parameter == "mean_value"]
2022-03-04 15:33:15.220 | DEBUG    | vivarium.framework.values:reg

2022-03-04 15:33:20.779 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline child_stunting.propensity
2022-03-04 15:33:20.781 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline child_stunting.exposure
2022-03-04 15:33:21.103 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.child_stunting.exposure_parameters
2022-03-04 15:33:22.028 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering diarrheal_diseases.incidence_rate.1.adjust_target as modifier to diarrheal_diseases.incidence_rate
2022-03-04 15:33:22.029 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering diarrheal_diseases.incidence_rate.paf.1.lookup_table_39 as modifier to diarrheal_diseases.incidence_rate.paf
2022-03-04 15:33:22.298 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering measles.incidence

2022-03-04 15:33:30.207 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering disability_weight.5.moderate_acute_malnutrition.disability_weight as modifier to disability_weight
2022-03-04 15:33:30.330 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition.excess_mortality_rate
2022-03-04 15:33:30.331 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition.excess_mortality_rate.paf
2022-03-04 15:33:30.332 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering mortality_rate.5.riskstate.moderate_acute_malnutrition.adjust_mortality_rate as modifier to mortality_rate
2022-03-04 15:33:30.852 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline moderate_acute_malnutrition_to_severe_acute_malnutrition.transition_rate
2022-03-04 15:33:30.853 | DEBUG    | vivariu

2022-03-04 15:33:36.728 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering affected_unmodeled.cause_specific_mortality_rate.paf.1.lookup_table_105 as modifier to affected_unmodeled.cause_specific_mortality_rate.paf
2022-03-04 15:33:36.747 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline maternal_malnutrition.propensity
2022-03-04 15:33:36.748 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline maternal_malnutrition.exposure
2022-03-04 15:33:37.036 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.maternal_malnutrition.exposure_parameters
2022-03-04 15:33:37.037 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.maternal_malnutrition.exposure_parameters.paf
2022-03-04 15:33:37.038 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Re

2022-03-04 15:33:41.095 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline x_factor.propensity
2022-03-04 15:33:41.096 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline x_factor.exposure
2022-03-04 15:33:41.248 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.x_factor.exposure_parameters
2022-03-04 15:33:41.250 | DEBUG    | vivarium.framework.values:_register_value_producer:325 - Registering value pipeline risk_factor.x_factor.exposure_parameters.paf
2022-03-04 15:33:41.393 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering mild_child_wasting_to_moderate_acute_malnutrition.transition_rate.2.adjust_target as modifier to mild_child_wasting_to_moderate_acute_malnutrition.transition_rate
2022-03-04 15:33:41.394 | DEBUG    | vivarium.framework.values:register_value_modifier:375 - Registering mild_child_wasting_

## Capture birthweight at initialization
For the early and late neonatal age groups

In [5]:
pop0 = sim.get_population()
pop0

,tracked,alive,sex,location,entrance_time,exit_time,age,cause_of_death,years_of_life_lost,years_lived_with_disability,...,effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_affected_unmodeled_relative_risk,effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_affected_unmodeled_relative_risk,severe_acute_malnutrition_event_time,severe_acute_malnutrition_event_count,moderate_acute_malnutrition_event_count,moderate_acute_malnutrition_event_time,mild_child_wasting_event_count,mild_child_wasting_event_time,susceptible_to_child_wasting_event_time,susceptible_to_child_wasting_event_count
0,True,alive,Male,Ethiopia,2021-06-30 12:00:00,NaT,0.019167,not_dead,0.0,0.0,...,9.637755,13.663443,NaT,0,0,NaT,0,2021-06-30 12:00:00,NaT,0
1,True,alive,Female,Ethiopia,2021-06-30 12:00:00,NaT,0.012411,not_dead,0.0,0.0,...,1.672641,1.279003,NaT,0,0,NaT,0,NaT,NaT,0
2,True,alive,Male,Ethiopia,2021-06-30 12:00:00,NaT,0.023611,not_dead,0.0,0.0,...,2.174342,1.634141,NaT,0,0,NaT,0,NaT,NaT,0
3,True,alive,Female,Ethiopia,2021-06-30 12:00:00,NaT,0.002792,not_dead,0.0,0.0,...,1.478018,1.065888,NaT,0,0,NaT,0,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4996,True,alive,Male,Ethiopia,2021-06-30 12:00:00,NaT,0.005132,not_dead,0.0,0.0,...,3.286529,2.563116,NaT,0,0,NaT,0,2021-06-30 12:00:00,NaT,0
4997,True,alive,Female,Ethiopia,2021-06-30 12:00:00,NaT,0.061908,not_dead,0.0,0.0,...,1.507811,1.184742,NaT,0,0,NaT,0,NaT,NaT,0
4998,True,alive,Male,Ethiopia,2021-06-30 12:00:00,NaT,0.008969,not_dead,0.0,0.0,...,20.095121,48.889560,NaT,0,0,NaT,0,2021-06-30 12:00:00,NaT,0
4999,True,alive,Male,Ethiopia,2021-06-30 12:00:00,NaT,0.073318,not_dead,0.0,0.0,...,3.815269,3.257671,NaT,0,0,NaT,0,NaT,NaT,0


In [6]:
art = Artifact('/ihme/costeffectiveness/artifacts/vivarium_ciff_sam/ethiopia.hdf',
               filter_terms=['year_start == 2019', f'age_end <= 5'])

In [7]:
interpolators = art.load(data_keys.LBWSG.RELATIVE_RISK_INTERPOLATOR)['draw_0'].reset_index()
interpolators = (
    # isolate RRs for target and drop non-neonatal age groups since they have RR == 1.0
    interpolators[interpolators['age_end'] < 0.5]
    .drop(columns=['age_end', 'year_start', 'year_end'])
    .set_index(['sex', 'draw_0'])
    .apply(lambda row: (metadata.AGE_GROUP.EARLY_NEONATAL_ID if row['age_start'] == 0.0
                        else metadata.AGE_GROUP.LATE_NEONATAL_ID), axis=1)
    .rename('age_group_id')
    .reset_index()
    .set_index(['sex', 'age_group_id'])
)['draw_0']

interpolators = interpolators.apply(lambda x: pickle.loads(bytes.fromhex(x)))
interpolators

sex     age_group_id
Female  2               <scipy.interpolate.fitpack2.RectBivariateSplin...
        3               <scipy.interpolate.fitpack2.RectBivariateSplin...
Male    2               <scipy.interpolate.fitpack2.RectBivariateSplin...
        3               <scipy.interpolate.fitpack2.RectBivariateSplin...
Name: draw_0, dtype: object

In [8]:
# inputs for interpolator function
is_tmrel = (sim.get_value('low_birth_weight_and_short_gestation.exposure')(pop0.index)
            .isin(data_keys.LBWSG.TMREL_CATEGORIES))
is_male = pop0['sex']=='Male'
is_enn = pop0['age']<7/365
gestational_age = sim.get_value('short_gestation.exposure')(pop0.index)
birth_weight = sim.get_value('low_birth_weight.exposure')(pop0.index)

AttributeError: '__LowBirthWeightShortGestation' object has no attribute 'TMREL_CATEGORIES'

In [ ]:
log_relative_risk = pd.Series(0.0, index=pop0.index, name='enn_interpolated_rr') #create series initialized at  0
log_relative_risk[is_male & ~is_tmrel] = (
    interpolators['Male', 2](gestational_age[is_male & ~is_tmrel],
                                            birth_weight[is_male & ~is_tmrel], grid=False)
)
log_relative_risk[~is_male & ~is_tmrel] = (
    interpolators['Female', 2](gestational_age[~is_male & ~is_tmrel],
                                              birth_weight[~is_male & ~is_tmrel], grid=False)
)
enn_rrs = np.exp(log_relative_risk)
enn_rrs

In [ ]:
log_relative_risk = pd.Series(0.0, index=pop0.index, name='lnn_interpolated_rr') #create series initialized at  0
log_relative_risk[is_male & ~is_tmrel] = (
    interpolators['Male', 3](gestational_age[is_male & ~is_tmrel],
                                            birth_weight[is_male & ~is_tmrel], grid=False)
)
log_relative_risk[~is_male & ~is_tmrel] = (
    interpolators['Female', 3](gestational_age[~is_male & ~is_tmrel],
                                              birth_weight[~is_male & ~is_tmrel], grid=False)
)
lnn_rrs = np.exp(log_relative_risk)

In [10]:
data = pd.concat([pop0.loc[:, ['age','sex',
                              'effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk',
                              'effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk']],
                 sim.get_value('low_birth_weight_and_short_gestation.exposure')(pop0.index),
                 sim.get_value('low_birth_weight.exposure')(pop0.index),
                 sim.get_value('effect_of_low_birth_weight_and_short_gestation_on_diarrheal_diseases.relative_risk')(pop0.index),
                 sim.get_value('short_gestation.exposure')(pop0.index)], 
                 axis=1).rename(columns={0:'relative_risk'})
#data = pd.concat([data,enn_rrs,lnn_rrs],axis=1)
#assert np.all(test.enn_interpolated_rr==test.effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_affected_unmodeled_relative_risk)
#assert np.all(test.lnn_interpolated_rr==test.effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_affected_unmodeled_relative_risk)
data

,age,sex,effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk,effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk,risk_factor.low_birth_weight_and_short_gestation.exposure,low_birth_weight_exposure,relative_risk,short_gestation_exposure
0,0.019167,Male,9.637755,13.663443,cat32,2128.068591,13.663443,39.890216
1,0.012411,Female,1.672641,1.279003,cat51,3032.995408,1.279003,41.852250
2,0.023611,Male,2.174342,1.634141,cat43,2932.382704,2.174342,41.059242
3,0.002792,Female,1.478018,1.065888,cat52,3159.153958,1.065888,39.706669
...,...,...,...,...,...,...,...,...
4996,0.005132,Male,3.286529,2.563116,cat44,2654.215222,2.563116,38.119966
4997,0.061908,Female,1.507811,1.184742,cat48,3449.273243,1.507811,37.471247
4998,0.008969,Male,20.095121,48.889560,cat29,1690.467032,48.889560,33.252966
4999,0.073318,Male,3.815269,3.257671,cat42,2582.238828,3.815269,37.870875


In [11]:
data_t = data.copy()
data['timestep'] = 0 
all_data = data.copy()
for x in list(range(1,15)):
    # take time step and initialize pop tables
    sim.step()
    data_t_minus_1 = data_t.copy()
    pop_t = sim.get_population()
    
    # interpolate rrs
    #is_tmrel = (sim.get_value('low_birth_weight_and_short_gestation.exposure')(pop_t.index)
    #            .isin(data_keys.LBWSG.TMREL_CATEGORIES))
    #is_male = pop_t['sex']=='Male'
    #is_enn = pop_t['age']<7/365
    gestational_age = sim.get_value('short_gestation.exposure')(pop_t.index)
    birth_weight = sim.get_value('low_birth_weight.exposure')(pop_t.index)
    #log_relative_risk = pd.Series(0.0, index=pop_t.index, name='enn_interpolated_rr') 
    #log_relative_risk[is_male & ~is_tmrel] = (
    #    interpolators['Male', 2](gestational_age[is_male & ~is_tmrel],
    #                                            birth_weight[is_male & ~is_tmrel], grid=False)
    #)
    #log_relative_risk[~is_male & ~is_tmrel] = (
    #    interpolators['Female', 2](gestational_age[~is_male & ~is_tmrel],
    #                                              birth_weight[~is_male & ~is_tmrel], grid=False)
    #)
    #enn_rrs = np.exp(log_relative_risk)
    #log_relative_risk = pd.Series(0.0, index=pop_t.index, name='lnn_interpolated_rr')
    #log_relative_risk[is_male & ~is_tmrel] = (
    #    interpolators['Male', 3](gestational_age[is_male & ~is_tmrel],
    #                                            birth_weight[is_male & ~is_tmrel], grid=False)
    #)
    #log_relative_risk[~is_male & ~is_tmrel] = (
    #    interpolators['Female', 3](gestational_age[~is_male & ~is_tmrel],
    #                                              birth_weight[~is_male & ~is_tmrel], grid=False)
    #)
    #lnn_rrs = np.exp(log_relative_risk)
    
    data_t = pd.concat([pop_t.loc[:, ['age','sex', 'alive', 'tracked', 'entrance_time', 'exit_time',
                              'effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk',
                              'effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk']],
                 sim.get_value('low_birth_weight_and_short_gestation.exposure')(pop_t.index),
                 sim.get_value('low_birth_weight.exposure')(pop_t.index),
                 sim.get_value('effect_of_low_birth_weight_and_short_gestation_on_diarrheal_diseases.relative_risk')(pop_t.index),
                 sim.get_value('short_gestation.exposure')(pop_t.index)], 
                 axis=1).rename(columns={0:'relative_risk'})
    #data_t = pd.concat([data_t,enn_rrs,lnn_rrs],axis=1)
    
    data_t['timestep'] = x
    all_data = pd.concat([all_data,data_t])


2022-03-04 15:39:56.707 | DEBUG    | vivarium.framework.engine:step:142 - 2021-07-01 00:00:00
/ihme/code/alibow/miniconda/envs/vivarium_ciff_sam/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
2022-03-04 15:40:10.935 | DEBUG    | vivarium.framework.engine:step:142 - 2021-07-01 12:00:00
2022-03-04 15:40:23.928 | DEBUG    | vivarium.framework.engine:step:142 - 2021-07-02 00:00:00
2022-03-04 15:40:36.916 | DEBUG    | vivarium.framework.engine:step:142 - 2021-07-02 12:00:00
2022-03-04 15:40:50.154 | DEBUG    | vivarium.framework.engine:step:142 - 2021-07-03 00:00:00
2022-03-04 15:41:03.348 | DEBUG    | vivarium.framework.engine:step:142 - 2021-07-03 12:00:00
2022-03-04 15:41:16.371 | DEBUG    | vivarium

In [12]:
all_data

,age,sex,effect_of_low_birth_weight_and_short_gestation_on_late_neonatal_diarrheal_diseases_relative_risk,effect_of_low_birth_weight_and_short_gestation_on_early_neonatal_diarrheal_diseases_relative_risk,risk_factor.low_birth_weight_and_short_gestation.exposure,low_birth_weight_exposure,relative_risk,short_gestation_exposure,timestep,alive,tracked,entrance_time,exit_time
0,0.019167,Male,9.637755,13.663443,cat32,2128.068591,13.663443,39.890216,0,NaN,NaN,NaT,NaT
1,0.012411,Female,1.672641,1.279003,cat51,3032.995408,1.279003,41.852250,0,NaN,NaN,NaT,NaT
2,0.023611,Male,2.174342,1.634141,cat43,2932.382704,2.174342,41.059242,0,NaN,NaN,NaT,NaT
3,0.002792,Female,1.478018,1.065888,cat52,3159.153958,1.065888,39.706669,0,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6212,0.000308,Male,2.462699,1.993749,cat42,2928.783200,1.993749,37.058440,14,alive,True,2021-07-07 12:00:00,NaT
6213,0.000080,Female,4.332689,3.386296,cat44,2513.384713,3.386296,38.222264,14,alive,True,2021-07-07 12:00:00,NaT
6214,0.001238,Female,1.102582,1.000000,cat51,3253.509786,1.000000,40.990160,14,alive,True,2021-07-07 12:00:00,NaT
6215,0.001270,Male,2.756995,2.033150,cat44,2712.863078,2.033150,39.054821,14,alive,True,2021-07-07 12:00:00,NaT


In [34]:
list(all_data.loc[(all_data.age<6/365)&(all_data.timestep==0)].index.unique())

[1,
 3,
 4,
 12,
 15,
 21,
 45,
 46,
 48,
 52,
 54,
 58,
 60,
 68,
 70,
 73,
 81,
 84,
 86,
 91,
 92,
 94,
 99,
 106,
 107,
 126,
 144,
 147,
 151,
 153,
 154,
 161,
 162,
 164,
 165,
 166,
 175,
 183,
 184,
 189,
 192,
 198,
 200,
 201,
 204,
 208,
 214,
 221,
 224,
 230,
 235,
 236,
 242,
 248,
 254,
 255,
 258,
 259,
 273,
 276,
 282,
 283,
 286,
 296,
 299,
 306,
 322,
 324,
 328,
 329,
 332,
 333,
 347,
 358,
 360,
 368,
 369,
 372,
 376,
 392,
 398,
 402,
 408,
 410,
 414,
 419,
 425,
 426,
 429,
 433,
 442,
 443,
 452,
 462,
 466,
 478,
 479,
 480,
 481,
 492,
 501,
 503,
 504,
 507,
 511,
 518,
 528,
 534,
 540,
 543,
 549,
 550,
 559,
 561,
 562,
 563,
 565,
 569,
 571,
 577,
 578,
 582,
 590,
 596,
 603,
 608,
 616,
 622,
 624,
 626,
 631,
 635,
 636,
 645,
 647,
 655,
 663,
 666,
 671,
 678,
 682,
 685,
 691,
 692,
 694,
 699,
 700,
 702,
 708,
 709,
 711,
 715,
 717,
 719,
 722,
 723,
 730,
 736,
 737,
 740,
 741,
 742,
 745,
 753,
 766,
 770,
 772,
 773,
 784,
 785,
 787,


In [35]:
for i in list(all_data.loc[(all_data.age<6/365)&(all_data.timestep==0)].index.unique()):
    assert len(all_data.loc[i].low_birth_weight_exposure.unique())==1